In [2]:
import pandas as pd
import pyodbc 
import numpy as np


In [3]:
#initial cleaning function
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0]) #use stock tickers as the index
    return adj


cc=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result data\{0} 2007-2017.xlsx'.format('Asia'),sheet_name='Index'))

In [4]:
def fundamental_df_clean(region,tabs,volume_low,size_low):
    data_dict={}

    for s in tabs:
        df=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result data\{0} 2007-2017.xlsx'.\
                                     format(region),sheet_name=s))
        data_dict[s]=df

    # select the right universe for US
    universe_evaluate=(data_dict["Market cap"]>size_low)&(data_dict["Volume"]>volume_low)
    data_dict["Revision"]=data_dict["Revision"][universe_evaluate]
    data_dict["Market cap"]=data_dict["Market cap"][data_dict["Market cap"]>size_low]    
    
    
    fundamental_df=data_dict["Date"].unstack(-1).to_frame()
    fundamental_df.index=fundamental_df.apply(lambda x: x.name[1],axis=1)
    fundamental_df.columns=["Date"]
    
    for i in tabs:
        try:
            fundamental_df[i]=data_dict[i].unstack(-1).values
        except ValueError:
            pass
        
    fundamental_df=fundamental_df.dropna(how="all",axis=0)
    fundamental_df=fundamental_df[-np.isnan(fundamental_df["Revision"])]
    fundamental_df=fundamental_df[-np.isnat(fundamental_df.Date)]
    if 'Property' in data_dict.keys():
        fundamental_df["Supersector"]=fundamental_df.apply(lambda x:data_dict["Property"].loc[x.name,"Supersector"]\
                                               if type(x.name)==str else None,axis=1)
    if 'Index' in data_dict.keys():
        fundamental_df["Index"]=fundamental_df.apply(lambda x: data_dict["Index"].loc[x.name].iloc[0] if type(x.name)==str\
                                                     else None,axis=1)
    
    fundamental_df=fundamental_df[fundamental_df.apply(lambda x: type(x.name)==str,axis=1)]
        
    return fundamental_df

In [25]:
US=fundamental_df_clean('US',['Date','Revision','Market cap','Volume','Revision vol','Property'],3,500)

In [5]:
whole_US=fundamental_df_clean('US',['Date','Revision','Market cap','Volume','Revision vol','Property'],0,0)

In [7]:
whole_US.to_csv("whole_US.csv")

In [27]:
Europe=fundamental_df_clean('Europe',['Date','Revision','Market cap','Volume','Revision vol','Property'],3,500)

In [45]:
Asia=fundamental_df_clean('Asia',['Date','Revision','Market cap','Volume'],3,500)

In [46]:
Asia

,Date,Revision,Market cap,Volume
000060 KS,2017-11-15,0.015214,2473.8353,4.676833
000080 KS,2017-11-14,-0.010933,1588.1563,4.943811
000100 KS,2017-10-31,-0.133019,2128.2039,9.686328
000120 KS,2017-11-08,-0.142718,3205.3041,5.439640
000150 KS,2017-11-06,0.257641,2460.2929,5.087403
000210 KS,2017-11-03,0.172044,2718.1398,9.627850
000270 KS,2017-10-27,-0.017287,12373.4431,34.798261
000660 KS,2017-10-26,0.049266,50889.3602,377.239534
000720 KS,2017-10-27,-0.104364,3922.7520,14.856339
000810 KS,2017-10-31,0.083693,11565.7913,16.752293


In [29]:
Europe.to_csv('Europe.csv')

In [47]:
Asia.to_csv('Asia.csv')